<a href="https://colab.research.google.com/github/Reichart88/endocrynology_base_neuro/blob/main/%D0%9F%D1%80%D0%BE%D1%81%D1%82%D0%BE%D0%B9_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%BE%D1%82%D1%80%D1%83%D0%B4%D0%BD%D0%B8%D0%BA_(%D0%B2%D1%80%D0%B0%D1%87).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00

In [ ]:
import getpass
import os
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
import requests
from openai import OpenAI
import gradio as gr
import tiktoken
import re

# Установка API ключа
os.environ["OPENAI_API_KEY"] = getpass.getpass("Paste your key:")

class GPT():
    def __init__(self, model="gpt-3.5-turbo"):
        self.log = ''
        self.model = model
        self.search_index = None
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        self.doc_url = "https://docs.google.com/document/d/16lVkyDUpmPGDH0Qu1_XfXQmDKY6DGmieVC82nkHvBGM/edit?usp=sharing"
        self.system_prompt = '''Ты опытный врач-эндокринолог с 15-летним стажем. Твоя задача - консультировать пациентов по вопросам эндокринологии, основываясь на предоставленной медицинской информации.

                    Стиль общения:
                    - Говори профессионально, но понятно для пациента
                    - Используй вежливый и спокойный тон
                    - Проявляй эмпатию

                    Важные правила:
                    1. Отвечай ТОЛЬКО на основе предоставленной информации из документа
                    2. Если информации недостаточно, обязательно скажи об этом и рекомендуй обратиться к врачу очно
                    3. НЕ ставь диагнозы
                    4. Давай только общие рекомендации из документа
                    5. При упоминании серьезных симптомов настаивай на очной консультации
                    6. Структурируй ответ по пунктам для лучшего восприятия
                    7. В конце каждого ответа напоминай, что это общая информация и необходима консультация врача'''

    def load_search_indexes(self):
        match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', self.doc_url)
        if match_ is None:
            raise ValueError('Неверный Google Docs URL')

        doc_id = match_.group(1)
        response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
        response.raise_for_status()

        return self.create_embedding(response.text)

    def num_tokens_from_string(self, string):
        encoding = tiktoken.encoding_for_model(self.model)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    def create_embedding(self, data):
        source_chunks = []
        splitter = CharacterTextSplitter(separator="\n",
                                       chunk_size=512,
                                       chunk_overlap=50)

        for chunk in splitter.split_text(data):
            source_chunks.append(Document(page_content=chunk, metadata={}))

        # Подсчет токенов
        count_token = self.num_tokens_from_string(' '.join([x.page_content for x in source_chunks]))
        self.log += f'Количество токенов в документе: {count_token}\n'

        self.search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings())
        self.log += f'Данные из документа загружены в векторную базу данных\n'

        return self.search_index

    def num_tokens_from_messages(self, messages, model):
        try:
            encoding = tiktoken.encoding_for_model(model)
        except KeyError:
            self.log += "Предупреждение: модель не создана. Используется cl100k_base кодировка.\n"
            encoding = tiktoken.get_encoding("cl100k_base")

        if "gpt-3.5-turbo" in model:
            tokens_per_message = 3
            tokens_per_name = 1
        else:
            tokens_per_message = 3
            tokens_per_name = 1

        num_tokens = 0
        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
        num_tokens += 3
        return num_tokens

    def answer_index(self, question, temp=0.7):
        if not self.search_index:
            self.log += 'Модель необходимо обучить!\n'
            return ''

        docs = self.search_index.similarity_search(question, k=7)
        self.log += 'Выбираем документы по степени схожести с вопросом:\n'

        context = '\n'.join([f'Отрывок документа №{i+1}:\n' + doc.page_content + '\n' for i, doc in enumerate(docs)])
        self.log += f'{context}\n'

        messages = [
            {"role": "system", "content": self.system_prompt + f"\n\nКонтекст:\n{context}"},
            {"role": "user", "content": question}
        ]

        self.log += f"\nТокенов использовано на вопрос: {self.num_tokens_from_messages(messages, self.model)}\n"

        completion = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=temp
        )

        self.log += '\nСтатистика по токенам:\n'
        self.log += f'Токенов на вопрос: {completion.usage.prompt_tokens}\n'
        self.log += f'Всего токенов: {completion.usage.total_tokens}\n'

        return completion.choices[0].message.content

# Инициализация GPT
gpt = GPT("gpt-3.5-turbo")

def train():
    try:
        gpt.log = ''  # Очищаем лог перед обучением
        gpt.load_search_indexes()
        return "Модель успешно обучена", ""  # Возвращаем статус и пустой лог
    except Exception as e:
        return f"Ошибка при обучении модели: {str(e)}", ""

def predict(question):
    if not gpt.search_index:
        return "Сначала необходимо обучить модель", "Модель не обучена"
    try:
        gpt.log = ''  # Очищаем лог перед новым предсказанием
        response = gpt.answer_index(question)
        return response, gpt.log
    except Exception as e:
        error_msg = f"Ошибка при получении ответа: {str(e)}"
        return error_msg, error_msg

# Интерфейс Gradio
with gr.Blocks() as demo:
    gr.Markdown("# Виртуальный эндокринолог")

    # Кнопка обучения во всю ширину
    train_btn = gr.Button("Обучить модель", size="lg")
    train_output = gr.Textbox(label="Статус обучения")

    # Поле ввода вопроса
    question = gr.Textbox(label="Ваш вопрос", placeholder="Задайте вопрос по эндокринологии...")

    # Кнопка получения ответа
    submit_btn = gr.Button("Получить ответ")

    # Поле вывода ответа
    answer = gr.Textbox(label="Ответ врача")

    # Поле логирования
    log = gr.Textbox(label="Логирование", lines=5)

    # Обработчики событий
    train_btn.click(train, outputs=[train_output, log])
    submit_btn.click(predict, inputs=question, outputs=[answer, log])

# Запуск приложения
demo.launch()



Paste your key:··········
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1f29147da73b85eac5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
